<a href="https://colab.research.google.com/github/sanaeaba/Recommende_system_coursera/blob/master/NeuFM_Coursera_without_contexts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fast-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from fast_ml.model_development import train_valid_test_split
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dropout, Dense, BatchNormalization,concatenate,Multiply,Dot
from tensorflow.keras import regularizers
#from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

In [ ]:
data = pd.read_csv("merged_coursera_datase.csv")
data

,course_id,user_id,rating,Sentiment,course_difficulty
0,289,8507,2,0,0
1,289,15643,2,1,0
2,289,3474,2,2,0
3,289,7221,3,0,0
4,289,8776,3,0,0
...,...,...,...,...,...
20117,210,1060,4,0,2
20118,210,4906,4,0,2
20119,210,838,4,2,2
20120,210,695,4,0,2


In [ ]:
from sklearn.preprocessing import LabelEncoder
course_encoder = LabelEncoder()
user_encoder = LabelEncoder()

data['course_id'] = course_encoder.fit_transform(data.course_id)
data['user_id'] = user_encoder.fit_transform(data.user_id)
data

,course_id,user_id,rating,Sentiment,course_difficulty
0,289,8507,2,0,0
1,289,15643,2,1,0
2,289,3474,2,2,0
3,289,7221,3,0,0
4,289,8776,3,0,0
...,...,...,...,...,...
20117,210,1060,4,0,2
20118,210,4906,4,0,2
20119,210,838,4,2,2
20120,210,695,4,0,2


In [ ]:
num_users = len(data.user_id.unique())
num_courses = len(data.course_id.unique())
#num_course_metadata = data.course_metadata.max()
#num_user_metadata = data.user_metadata.max()
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(data, target = 'rating', train_size=0.7, valid_size=0.1, test_size=0.2)

In [ ]:
latent_dim = 18

# Define inputs
course_input = Input(shape=[1], name='course-input')
user_input = Input(shape=[1], name='user-input')

# MF Embeddings (only using MF component)
course_embedding_mf = Embedding(num_courses + 1, latent_dim, name='course-embedding-mf')(course_input)
course_vec_mf = Flatten(name='flatten-course-mf')(course_embedding_mf)

user_embedding_mf = Embedding(num_users + 1, latent_dim, name='user-embedding-mf')(user_input)
user_vec_mf = Flatten(name='flatten-user-mf')(user_embedding_mf)

# MF dropout with a higher rate
user_vec_mf = Dropout(0.5)(user_vec_mf)
course_vec_mf = Dropout(0.5)(course_vec_mf)

# Prediction from MF component
pred_mf = Multiply()([course_vec_mf, user_vec_mf])

# Final prediction
result = Dense(1, name='result', activation='linear')(pred_mf)

model = Model([user_input, course_input], result)
model.compile(optimizer=keras.optimizers.Adam(lr=0.0005), loss='mean_squared_error')

In [ ]:
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 course-input (InputLayer)   [(None, 1)]                  0         []                            
                                                                                                  
 user-input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 course-embedding-mf (Embed  (None, 1, 18)                5706      ['course-input[0][0]']        
 ding)                                                                                            
                                                                                                  
 user-embedding-mf (Embeddi  (None, 1, 18)                324180    ['user-input[0][0]']   

In [ ]:
from sklearn.metrics import mean_absolute_error
history = model.fit([X_train.user_id, X_train.course_id], y_train, epochs=65, batch_size=512, validation_data=([X_valid.user_id, X_valid.course_id], y_valid))
y_hat = model.predict([X_test.user_id, X_test.course_id])
y_true = y_test

Epoch 1/65
28/28 [==============================] - 2s 24ms/step - loss: 21.1775 - val_loss: 21.0317
Epoch 2/65
28/28 [==============================] - 0s 15ms/step - loss: 20.9247 - val_loss: 20.7817
Epoch 3/65
28/28 [==============================] - 0s 13ms/step - loss: 20.6729 - val_loss: 20.5341
Epoch 4/65
28/28 [==============================] - 0s 15ms/step - loss: 20.4217 - val_loss: 20.2887
Epoch 5/65
28/28 [==============================] - 0s 14ms/step - loss: 20.1698 - val_loss: 20.0456
Epoch 6/65
28/28 [==============================] - 0s 14ms/step - loss: 19.9133 - val_loss: 19.8042
Epoch 7/65
28/28 [==============================] - 0s 15ms/step - loss: 19.6479 - val_loss: 19.5645
Epoch 8/65
28/28 [==============================] - 0s 13ms/step - loss: 19.3693 - val_loss: 19.3261
Epoch 9/65
28/28 [==============================] - 0s 14ms/step - loss: 19.0709 - val_loss: 19.0883
Epoch 10/65
28/28 [==============================] - 0s 13ms/step - loss: 18.7463 - val_los

In [ ]:
!pip install recmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.12.2
    Uninstalling seaborn-0.12.2:
      Successfully uninstalled seaborn-0.12.2
ERROR: pip's dependency resolver does

# **--------------after--------------**

In [ ]:
import recmetrics
MAE = mean_absolute_error(y_true, y_hat)
print("MAE: ", MAE)
print("MSE: ", recmetrics.mse(y_true,y_hat))
print("RMSE: ", recmetrics.rmse(y_true,y_hat))

MAE:  3.372480045401532
MSE:  12.516930183938666
RMSE:  3.537927385339991
